# Task- 3: Multilabel Classification of Counter Speech
### This notebook is used to measure performance of the combination of different classifier and different feature engineering techniques  used in the project

In [17]:
from commen_preprocess import *
from skmultilearn.adapt import brknn
from skmultilearn.adapt import MLkNN
from skmultilearn.problem_transform import br
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.neurofuzzy import MLARAM
import scipy
import sklearn.metrics
import argparse
import numpy
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import neighbors
from sklearn import ensemble
from sklearn import neural_network
from sklearn import linear_model
import gensim, sklearn
from collections import defaultdict
#from batch_gen import batch_gen
from nltk.tokenize import TweetTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from xgboost import XGBClassifier
from sklearn.externals import joblib
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import VotingClassifier
import os
from string import punctuation
from gensim.parsing.preprocessing import STOPWORDS
import tensorflow as tf
import tensorflow_hub as hub
import re
from tqdm import tqdm
import pickle
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
from sklearn.metrics import confusion_matrix,make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
ps = PorterStemmer()
import pandas as pd
import numpy as np
import json
from catboost import CatBoostClassifier
from scipy.sparse import vstack, hstack
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import os


import sys
sys.path.insert(0, '../')
####features module has the necessary function for feature generation 
from utils.multi_features import *
###tokenize module has the tokenization funciton
from utils.tokenize import *
###helper prints confusion  matrix and stores results
from utils.helper import *
###common preprocessing imports
from utils.commen_preprocess import *

import joblib

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Word_to_vec model  loading 
1. change the path of glove model file

In [3]:
GLOVE_MODEL_FILE="../../embeddings/glove.840B.300d.txt"
print(os.path.isfile(GLOVE_MODEL_FILE))

## change the embedding dimension according to the model
EMBEDDING_DIM = 300
def loadGloveModel2(glove_file):
    tmp_file = get_tmpfile("test_crawl_200.txt")

    # call glove2word2vec script
    # default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>

    glove2word2vec(glove_file, tmp_file)
    model=KeyedVectors.load_word2vec_format(tmp_file)
    return model

word2vec_model = loadGloveModel2(GLOVE_MODEL_FILE)

True


### Dataset is loaded here

In [4]:
### set the path of the the json file where to load the data 
path='../Data/Counterspeech_Dataset.json'
with open(path) as fp:
    train_data = json.load(fp)

In [5]:
### return label
def convert_class_label(input_text):
    if input_text:
        return 'counter'
    else:
        return 'noncounter'

#### Load the data in the dataframe  having the four fields as
1. id
2. class
3. community
4. category(labels)
5. text

In [6]:
pd_train = pd.DataFrame(columns=['id','class','community','category','text'])

for count, each in enumerate(train_data):
    try:
        pd_train.loc[count]  = [each['id'], convert_class_label(each['CounterSpeech']), each['Community'],each['Category'],each['commentText']]
    except:
        pass
print('Training Data Loading Completed...') 

### select the data having the labels ...default denotes the non counter speech
pd_train =pd_train[pd_train['category']!='Default']

Training Data Loading Completed...


#### Extraction of the category column into a multi-hot vector encoding 

In [7]:
list1=[[],[],[],[],[],[],[],[],[],[]]
for ele in pd_train['category']:
    temp=[]
    if type(ele) is int:
        ele =str(ele)
    for i in range(0,len(ele),2):
        temp.append(ord(ele[i])-ord('0'))
    #print(temp)
    if(len(temp)==0):
        print(temp)
    for i in range(0,10):
        if i+1 in temp:
            list1[i].append(1)
        else:
            list1[i].append(0)
y_train=np.array([np.array(xi) for xi in list1])            

In [8]:
### final dataframe for the task created 
pd_train = pd.DataFrame({'text':list(pd_train['text']),'cat0':list1[0],'cat1':list1[1],'cat2':list1[2],'cat3':list1[3],'cat4':list1[4],'cat5':list1[5],'cat6':list1[6],'cat7':list1[7],'cat8':list1[8],'cat9':list1[9]})
### drop the entries having blank entries
pd_train['text'].replace('', np.nan, inplace=True)
pd_train.dropna(subset=['text'], inplace=True)


In [9]:
pd_train.head()

cat0  cat1  cat2  cat3  cat4  cat5  cat6  cat7  cat8  cat9  \
0     0     1     0     0     0     0     0     1     0     0   
1     0     0     0     0     0     0     0     1     0     0   
2     0     0     0     0     0     0     0     1     0     0   
3     0     0     0     0     0     0     0     0     1     0   
4     0     1     0     0     0     0     0     0     0     0   

                                                text  
0  What kind of God is it that Hates people & Cur...  
1  Question - Which comment is racist and why?\n ...  
2  That Israel fail that is nothing new. That Isr...  
3  This god damned ignorant little prick doesn't ...  
4  our shit may be cooler but... police brutality...

In [10]:
#### converting the data into text and labels dictionary
def get_data():
    comments=pd_train['text'].values
    df = pd_train.drop('text', 1)
    labels=df.values
    list_comment=[]
    for comment,label in zip(comments,labels):
        temp={}
        temp['text']=comment
        temp['label']=label
        list_comment.append(temp)
    return list_comment    

### Different feature generation methods


### Main Classification function

In [11]:
import json
with open('multilabel_all_parameters.json') as f:
        parameters=json.load(f)


In [23]:
def classification_model(model_name,featureset_name,model_type=None,save_model=False):
    X,Y=get_feature(pd_train,featureset_name)
    model=get_model(model_type)
    try:
        model_parameter=parameters[classifier_model+'+'+feature_model]
        for k,v in param_set.items():
             setattr(model_parameter,k,v)
    except:
        pass
    
    
    if(model==None):
        return 1
    path=model_name+'_'+featureset_name
    os.makedirs(path, exist_ok=True)
    mskf = MultilabelStratifiedKFold(n_splits=10, random_state=0)
    X = numpy.array(X)
    Y = numpy.array(Y)
    
    
    if(save_model==True):
        Classifier=model
        Classifier.fit(X, Y)
        filename = '../Best_model/'+model_name+'_task_3.joblib.pkl'
        joblib.dump(Classifier, filename, compress=9)
    
    else:
        ham_list=[]
        acc_list=[]
        pre_list=[]
        rec_list=[]
        f1_list=[]
        hard_metric_list=[]
        hard_train_list=[]
        y_total_preds=[] 
        y_total=[]
        count=1
        for train_index, test_index in mskf.split(X, Y):
                #print("TRAIN:", train_index, "TEST:", test_index)
                print("crossval step--",str(count))
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = Y[train_index], Y[test_index]    
                clf = model
                clf.fit(X_train, y_train)
                y_pred =  clf.predict(X_test)
                y_train_pred =  clf.predict(X_train)

                if(scipy.sparse.issparse(y_pred)):
                   ham,acc,pre,rec,f1=calculate_score(y_test,y_pred.toarray())
                   for ele in y_test:
                      y_total.append(ele)
                   for ele in y_pred.toarray():
                      y_total_preds.append(ele)
                   accuracy_train=accuracy_score(y_train,y_train_pred.toarray())
                   accuracy_test=accuracy_score(y_test,y_pred.toarray())

                else:
                   ham,acc,pre,rec,f1=calculate_score(y_test,y_pred)
                   for ele in y_test:
                      y_total.append(ele)
                   for ele in y_pred:
                      y_total_preds.append(ele)
                   accuracy_train=my_accuracy_score(y_train,y_train_pred)
                   accuracy_test=my_accuracy_score(y_test,y_pred)

                ham_list.append(ham)
                acc_list.append(acc)
                pre_list.append(pre)
                rec_list.append(rec)
                f1_list.append(f1)
                hard_train_list.append(accuracy_train)
                hard_metric_list.append(accuracy_test)
                count=count+1
        y_total_preds=np.array(y_total_preds) 
        y_total=np.array(y_total)
        with open('all_preds_multilabel.pkl', 'wb') as f:
            pickle.dump([y_total,y_total_preds], f)

        for i in range(10):
            df_result=pandas_classification_report(y_total[:,i],y_total_preds[:,i])
            df_result.to_csv(path+'/report'+str(i)+'.csv')


        f = open(path+'/final_report.txt', "w")
        f.write(model_name)    
        f.write("The hard train score is :- " + str(numpy.mean(hard_train_list)))
        f.write("The hard metric score is :- " + str(numpy.mean(hard_metric_list)))
        f.write("The accuracy is :- " + str(numpy.mean(acc_list)))
        f.write("The precision is :- " + str(numpy.mean(pre_list)))
        f.write("The recall is :- " + str(numpy.mean(rec_list)))
        f.write("The f1_score is :- " + str(numpy.mean(f1_list)))
        f.write("The hamming loss is :-" + str(numpy.mean(ham_list)))
        f.close()


In [19]:
def get_model(m_type=None):
    if not m_type:
        print("ERROR: Please specify a model type!")
        return None
    if m_type == 'decision_tree_classifier':
        logreg = tree.DecisionTreeClassifier(class_weight='balanced')
    elif m_type == 'MLPClassifier':
        logreg = neural_network.MLPClassifier((500))
    elif m_type == 'KNeighborsClassifier':
        logreg = neighbors.KNeighborsClassifier(n_neighbors = 10)
    elif m_type == 'ExtraTreeClassifier':
        logreg = tree.ExtraTreeClassifier()
    elif m_type == 'ExtraTreeClassifier_2':
        logreg = ensemble.ExtraTreesClassifier()
    elif m_type == 'Logistic_Regression':
        logreg = OneVsRestClassifier(linear_model.LogisticRegression(class_weight='balanced'))
    elif m_type == 'RandomForestClassifier':
        logreg = ensemble.RandomForestClassifier(class_weight='balanced')
    elif m_type == 'binary_relevance_GaussianNB':
        logreg = OneVsRestClassifier(GaussianNB())
    elif m_type == 'SVC':
        logreg = OneVsRestClassifier(SVC(class_weight='balanced'))
    elif m_type == 'XG_BOOST':
        ###best_model####
        logreg = OneVsRestClassifier(XGBClassifier(eval_metric='logloss',objective= 'binary:logistic', nthread=12, scale_pos_weight=5))
    elif m_type == 'Catboost':
        logreg = OneVsRestClassifier(CatBoostClassifier(iterations=100,silent=True,scale_pos_weight=5))
    elif m_type == 'MLKNN':
        logreg = MLkNN(k=8)
    elif m_type == 'MLARAM':
        logreg = MLARAM(threshold=0.05, vigilance=0.95)
    else:
        print("ERROR: Please specify a correct model")
        return None

    return logreg


def get_feature(pd_train,f_type=None):
    if not f_type:
        print("ERROR: Please specify a model type!")
        return None,None
    if f_type == 'google_not_preprocess':
        X,y=gen_data_google2(pd_train)
    elif f_type == 'word_to_vec_embed':
        X,y=gen_data_embed(pd_train,word2vec_model)
    elif f_type == 'google_preprocess':
        X,y=gen_data_google(pd_train)
    elif f_type == 'tfidf_not_preprocess':
        X,y=gen_data_new_tfidf2(pd_train)
    elif f_type == 'tfidf_preprocess':
        X,y=gen_data_new_tfidf(pd_train)
    elif f_type == 'google_preprocess_tfidf_preprocess':
        X,y=combine_tf_google_rem(pd_train)
    elif f_type == 'google_nopreprocess_tfidf_nopreprocess':
        X,y=combine_tf_google_norem(pd_train)
    elif f_type == 'google_preprocess_tfidf_nopreprocess':
        X,y=combine_tf_norem_google_rem(pd_train)
    elif f_type == 'google_nopreprocess_tfidf_preprocess':
        X,y=combine_tf_rem_google_norimportem(pd_train)
    elif f_type == 'google_preprocess_embed':
        X,y=combine_google_rem_embed(pd_train,word2vec_model)
    elif f_type == 'tfidf_preprocess_embed':
        X,y=combine_tf_rem_embed(pd_train,word2vec_model)
    elif f_type == 'google_preprocess_tfidf_preprocess_embed':
        ###best features####
        X,y=combine_tf_rem_google_rem_embed(pd_train,word2vec_model)
    else:
        print("give correct feature selection")    
    return X,y



## Models available
1. decision_tree_classifier
2. MLPClassifier
3. KNeighborsClassifier
4. ExtraTreeClassifier
5. ExtraTreeClassifier_2
6. RandomForestClassifier
7. SVC
8. CatboostClassifier
9. XGB_classifier
10. Logistic_Regression
11. MLKNN
12. MLARAM
13. Gaussian Naive Bayes




## Feature Models available
1. google_not_preprocess
2. word_to_vec_embed
3. google_preprocess
4. tfidf_not_preprocess
5. tfidf_preprocess
6. google_preprocess_tfidf_preprocess
7. google_nopreprocess_tfidf_nopreprocess
8. google_preprocess_tfidf_nopreprocess
9. google_nopreprocess_tfidf_preprocess
10. google_preprocess_embed
11. tfidf_preprocess_embed
12. google_preprocess_tfidf_preprocess_embed


### Specify the model and the feature selection method

In [14]:
options_ft=['google_not_preprocess','word_to_vec_embed','google_preprocess','tfidf_not_preprocess','tfidf_preprocess','google_preprocess_tfidf_preprocess','google_nopreprocess_tfidf_nopreprocess','google_preprocess_tfidf_nopreprocess', 'google_nopreprocess_tfidf_preprocess','google_preprocess_embed','tfidf_preprocess_embed','google_preprocess_tfidf_preprocess_embed']
ft= widgets.Dropdown(options=options_ft, value=None)
print('select feature combination') 
ft

select feature combination


Dropdown(options=('google_not_preprocess', 'word_to_vec_embed', 'google_preprocess', 'tfidf_not_preprocess', 'tfidf_preprocess', 'google_preprocess_tfidf_preprocess', 'google_nopreprocess_tfidf_nopreprocess', 'google_preprocess_tfidf_nopreprocess', 'google_nopreprocess_tfidf_preprocess', 'google_preprocess_embed', 'tfidf_preprocess_embed', 'google_preprocess_tfidf_preprocess_embed'), value=None)

In [15]:
options_clf=['decision_tree_classifier','MLPClassifier','KNeighborsClassifier','ExtraTreeClassifier','ExtraTreeClassifier_2','RandomForestClassifier','SVC','Catboost','XGB_classifier','Logistic_Regression','Gaussian Naive bayes','MLKNN','MLARAM']
clf= widgets.Dropdown(options=options_clf, value=None)
print('select a model') 
clf

select a model


Dropdown(options=('decision_tree_classifier', 'MLPClassifier', 'KNeighborsClassifier', 'ExtraTreeClassifier', 'ExtraTreeClassifier_2', 'RandomForestClassifier', 'SVC', 'Catboost', 'XGB_classifier', 'Logistic_Regression', 'Gaussian Naive bayes', 'MLKNN', 'MLARAM'), value=None)

In [24]:
###specify the model name
abv_name=clf.value
###specify the feature model###
feature_select=ft.value
#### model name ####
model_name=clf.value

### calling of the method####
classification_model(abv_name,feature_select,model_name,save_model=True)


ending


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['fifti'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home